Aseem and I discussed how emission probabilities are to be calculated. We both had differing formulas. He shared his correspondence with you(Professor) on the same and we used the understanding that you provided in the same.

In [1]:
import nltk
from nltk.corpus import brown
from nltk.probability import FreqDist
import numpy as np
from tqdm import tqdm

#nltk.download('brown')
#nltk.download('universal_tagset')

In [2]:
# Calculate starting probabilities
# This is provided in the homework; no need to change this

brown_sents = brown.tagged_sents(tagset='universal')
starting_pos = [sent[0][1] for sent in brown_sents]
freq_pos = FreqDist(starting_pos)
for item in freq_pos:
  freq_pos[item] = freq_pos[item] / len(starting_pos)

In [3]:
# TODO: Calculate emission probabilities here
# Remember to ignore case and treat "Flies" the same as "flies"

brown_words = brown.tagged_words(tagset='universal')

In [4]:
flies=[]
fly=[]
for i,j in tqdm(brown_words):
    if i.lower()=='flies':
        flies.append(j)
    elif i.lower()=='fly':
        fly.append(j)

100%|████████████████████████████████████████████████████████████████████| 1161192/1161192 [00:01<00:00, 608388.67it/s]


In the Brown corpus with Universal tags, what is the set of part-of-speech tags we
observe as labels for the words in our sentence: “flies” and “fly”?

In [5]:
print('For fly labels are:\n',set(fly))
print('For flies labels are:\n',set(flies))

For fly labels are:
 {'NOUN', 'VERB'}
For flies labels are:
 {'NOUN', 'VERB'}


In [6]:
second_values = [tup[1] for tup in brown_words]
unq=set(second_values)

In [7]:
unq

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [8]:
total_counts={}
for i in unq:
    total_counts[i]=0
for i,j in tqdm(brown_words):
    total_counts[j]=total_counts[j]+1

100%|████████████████████████████████████████████████████████████████████| 1161192/1161192 [00:02<00:00, 528611.62it/s]


In [9]:
total_counts

{'X': 1386,
 'PRT': 29829,
 'DET': 137019,
 'NOUN': 275558,
 'ADJ': 83721,
 'PRON': 49334,
 'NUM': 14874,
 'ADP': 144766,
 'CONJ': 38151,
 'ADV': 56239,
 '.': 147565,
 'VERB': 182750}

In [10]:
elim={'fly':{},'flies':{}}
for i in elim:
    for j in unq:
        elim[i][j]=int(0)

In [11]:
elim

{'fly': {'X': 0,
  'PRT': 0,
  'DET': 0,
  'NOUN': 0,
  'ADJ': 0,
  'PRON': 0,
  'NUM': 0,
  'ADP': 0,
  'CONJ': 0,
  'ADV': 0,
  '.': 0,
  'VERB': 0},
 'flies': {'X': 0,
  'PRT': 0,
  'DET': 0,
  'NOUN': 0,
  'ADJ': 0,
  'PRON': 0,
  'NUM': 0,
  'ADP': 0,
  'CONJ': 0,
  'ADV': 0,
  '.': 0,
  'VERB': 0}}

In [12]:
for i,j in tqdm(brown_words):
    if i.lower()=='fly':
        elim['fly'][j]=(elim['fly'][j]+1)
    elif i.lower()=='flies':
        elim['flies'][j]=(elim['flies'][j]+1)

100%|████████████████████████████████████████████████████████████████████| 1161192/1161192 [00:02<00:00, 517490.71it/s]


In [13]:
for k1,subdict in tqdm(elim.items()):
    for k2,value in subdict.items():
        total=total_counts[k2]
        subdict[k2]=value/total

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


Provide all non-zero emission probabilities for “flies” and “fly” based on the tagged Brown
corpus. (Hint: refer to your answer to #1). Remember to ignore case (i.e., treat “Flies”
and “flies” as the same word).

In [14]:
for key,subdict in elim.items():
    print(key)
    for subkey,value in subdict.items():
        if value!=0:
            print(subkey,':',value)
            

fly
NOUN : 5.443500097983002e-05
VERB : 9.849521203830369e-05
flies
NOUN : 2.903200052257601e-05
VERB : 2.188782489740082e-05


Give an example of an emission probability (for ‘flies’ or ‘fly’) that is zero.

In [15]:
for key,subdict in elim.items():
    print(key)
    for subkey,value in subdict.items():
        if value==0:
            print(subkey,':',value)
            break

fly
X : 0.0
flies
X : 0.0


In [16]:
trans={'NOUN':{'NOUN':0,'VERB':0},'VERB':{'NOUN':0,'VERB':0}}
word_tag_pairs = list(nltk.bigrams(brown_words))
for ((word1,pos1),(word2,pos2)) in tqdm(word_tag_pairs):
    if pos1 in ('NOUN','VERB') and pos2 in('NOUN','VERB'):
        trans[pos1][pos2]=trans[pos1][pos2]+1
trans

100%|███████████████████████████████████████████████████████████████████| 1161191/1161191 [00:00<00:00, 3658213.43it/s]


{'NOUN': {'NOUN': 41309, 'VERB': 43819},
 'VERB': {'NOUN': 17851, 'VERB': 33672}}

In [17]:
for k1,subdict in tqdm(trans.items()):
    total=total_counts[k1]
    for k2,value in subdict.items():
        subdict[k2]=value/total
trans

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


{'NOUN': {'NOUN': 0.14991036369838653, 'VERB': 0.15901915386234478},
 'VERB': {'NOUN': 0.09767989056087552, 'VERB': 0.18425170998632012}}

Normally, you would want to calculate the transition probability matrix between all pairs
of possible states (i.e., parts-of-speech). Given that there are 12 parts-of-speech in the
universal tagset, this would involve creating a 12x12 matrix. However, for this problem, it
is sufficient to create a 2x2 matrix. Provide that transition probability matrix and its
values, and explain why no other transition probabilities are needed in order to correctly
tag the sentence “Flies fly”.

In [18]:
for k1,subdict in trans.items():
    for k2,val in subdict.items():
        print('P(',k2,'|',k1,')=',val)

P( NOUN | NOUN )= 0.14991036369838653
P( VERB | NOUN )= 0.15901915386234478
P( NOUN | VERB )= 0.09767989056087552
P( VERB | VERB )= 0.18425170998632012


We do not need other transition probabilities as for words 'flies' and 'fly' exist in two states only NOUN and VERB and therefore while using the Viterbi algorithm, the probabilities get multiplied by the emission probability of the word existing in that state, as these two words do not exist in any state other than NOUN or VERB the final product would always be 0 for other states.

Walk through the Viterbi algorithm to show how the sentence “Flies fly” would be tagged.
Your answer can take the form of a 2x2 matrix where the rows correspond to the parts of
speech that “flies” and “fly” can take (hint: refer to #1 and #2) and the columns
correspond to each word in the to-be-tagged sentence.

In [19]:
#Start Probabilities taken from table provided 
p_start={'DET':0.21,
'PRON':0.16,
'NOUN':0.14,
'ADP':0.12,
'ADV':0.09,
'.':0.09,
'CONJ':0.05,
'VERB':0.05,
'PRT':0.04,
'ADJ':0.03,
'NUM':0.02,
'X':0.0}
p_start

{'DET': 0.21,
 'PRON': 0.16,
 'NOUN': 0.14,
 'ADP': 0.12,
 'ADV': 0.09,
 '.': 0.09,
 'CONJ': 0.05,
 'VERB': 0.05,
 'PRT': 0.04,
 'ADJ': 0.03,
 'NUM': 0.02,
 'X': 0.0}

In [20]:
v={'NOUN':{'flies':0,'fly':0},'VERB':{'flies':0,'fly':0}}
v

{'NOUN': {'flies': 0, 'fly': 0}, 'VERB': {'flies': 0, 'fly': 0}}

In [21]:
v['NOUN']['flies']=p_start['NOUN']*elim['flies']['NOUN']
v['VERB']['flies']=p_start['VERB']*elim['flies']['VERB']

In [22]:
v

{'NOUN': {'flies': 4.064480073160642e-06, 'fly': 0},
 'VERB': {'flies': 1.094391244870041e-06, 'fly': 0}}

In [23]:
fly_NOUN_NOUN=v['NOUN']['flies']*trans['NOUN']['NOUN']*elim['fly']['NOUN']
print(fly_NOUN_NOUN)

3.316766448510059e-11


In [24]:
fly_VERB_NOUN=v['VERB']['flies']*trans['VERB']['NOUN']*elim['fly']['NOUN']
print(fly_VERB_NOUN)

5.8191025317547984e-12


In [25]:
v['NOUN']['fly']=max(fly_NOUN_NOUN,fly_VERB_NOUN)
v

{'NOUN': {'flies': 4.064480073160642e-06, 'fly': 3.316766448510059e-11},
 'VERB': {'flies': 1.094391244870041e-06, 'fly': 0}}

In [26]:
fly_NOUN_VERB=v['NOUN']['flies']*trans['NOUN']['VERB']*elim['fly']['VERB']
print(fly_NOUN_VERB)

6.366042833509492e-11


In [27]:
fly_VERB_VERB=v['VERB']['flies']*trans['VERB']['VERB']*elim['fly']['VERB']
print(fly_VERB_VERB)

1.9860915177589753e-11


In [28]:
v['VERB']['fly']=max(fly_NOUN_VERB,fly_VERB_VERB)
v

{'NOUN': {'flies': 4.064480073160642e-06, 'fly': 3.316766448510059e-11},
 'VERB': {'flies': 1.094391244870041e-06, 'fly': 6.366042833509492e-11}}

In [29]:
max(v['VERB']['fly'],v['NOUN']['fly'])

6.366042833509492e-11

As we can observe for 'fly' the 'VERB' state has the maximum probability. And we also know that the transition from 'NOUN'->'VERB' has a larger probability, therefore, the state of 'flies' is 'NOUN'. <br>
The POS tagging of 'Flies fly' is 'NOUN VERB'

Make a change to the emission probabilities, transition probabilities, or starting
probabilities (or any combination thereof) in such a way that it changes how the
sentence would be tagged (but make sure all probabilities that should sum to 1 still do).
Provide your change and what the resulting part of speech tags would be. Explain the
intuition for why your change to the probabilities causes a change in tagging.

Making changes to the starting Probabilities P(STATE|START) such that P(VERB|START) is significatinly higher than P(NOUN|START)

In [30]:
p_start={'DET':0.21,
'PRON':0.16,
'NOUN':0.14,
'ADP':0,
'ADV':0,
'.':0,
'CONJ':0,
'VERB':0.5,
'PRT':0,
'ADJ':0,
'NUM':0,
'X':0}
p_start

{'DET': 0.21,
 'PRON': 0.16,
 'NOUN': 0.14,
 'ADP': 0,
 'ADV': 0,
 '.': 0,
 'CONJ': 0,
 'VERB': 0.5,
 'PRT': 0,
 'ADJ': 0,
 'NUM': 0,
 'X': 0}

In [31]:
v={'NOUN':{'flies':0,'fly':0},'VERB':{'flies':0,'fly':0}}
v['NOUN']['flies']=p_start['NOUN']*elim['flies']['NOUN']
v['VERB']['flies']=p_start['VERB']*elim['flies']['VERB']
v


{'NOUN': {'flies': 4.064480073160642e-06, 'fly': 0},
 'VERB': {'flies': 1.094391244870041e-05, 'fly': 0}}

In [32]:
fly_NOUN_NOUN=v['NOUN']['flies']*trans['NOUN']['NOUN']*elim['fly']['NOUN']
print(fly_NOUN_NOUN)
fly_VERB_NOUN=v['VERB']['flies']*trans['VERB']['NOUN']*elim['fly']['NOUN']
print(fly_VERB_NOUN)
v['NOUN']['fly']=max(fly_NOUN_NOUN,fly_VERB_NOUN)
v

3.316766448510059e-11
5.8191025317547986e-11


{'NOUN': {'flies': 4.064480073160642e-06, 'fly': 5.8191025317547986e-11},
 'VERB': {'flies': 1.094391244870041e-05, 'fly': 0}}

In [33]:
fly_NOUN_VERB=v['NOUN']['flies']*trans['NOUN']['VERB']*elim['fly']['VERB']
print(fly_NOUN_VERB)
fly_VERB_VERB=v['VERB']['flies']*trans['VERB']['VERB']*elim['fly']['VERB']
print(fly_VERB_VERB)
v['VERB']['fly']=max(fly_NOUN_VERB,fly_VERB_VERB)
v

6.366042833509492e-11
1.9860915177589754e-10


{'NOUN': {'flies': 4.064480073160642e-06, 'fly': 5.8191025317547986e-11},
 'VERB': {'flies': 1.094391244870041e-05, 'fly': 1.9860915177589754e-10}}

Now the POS tagging of 'Flies fly' is 'VERB VERB'

As the Initial Cells attribute significantly towards the final Probability of a state, I assumed if I increased P(VERB|START) significantly the POS Tagging of 'Flies' would change from 'NOUN' to 'VERB' keeping everything else the same. And as I had assumed the same happened.

Calc for the Probabilities of Part 2 Q4

In [34]:
(0.8*0.45*0.2*0.9)*(0.1*0.7*0.1*0.2*0.1)

9.072000000000002e-06

In [35]:
(0.8*0.3*0.45*0.9)*(0.1*0.7*0.1*0.2*0.1)

1.3608e-05